In [ ]:
import networkx as nx
import numpy as np

file_path = "Vickers-Chan-7thGraders_multiplex.edges"
with open(file_path, 'r') as file:
    lines = file.readlines()

layer_graphs = {}
G = nx.MultiGraph()
for line in lines:
    layer, node1, node2, weight = map(int, line.strip().split())
    G.add_node(node1, layer=layer)
    G.add_node(node2, layer=layer)
    G.add_edge(node1, node2, layer=layer, weight=weight)
    if layer not in layer_graphs:
        layer_graphs[layer] = nx.MultiGraph()
    layer_graphs[layer].add_edge(node1, node2, weight=weight)

node_degrees = {}
for layer, layer_graph in layer_graphs.items():
    degrees = dict(layer_graph.degree())
    node_degrees[layer] = degrees

average_node_degrees = {}
normalized_node_degree_counts = []
for layer, degrees in node_degrees.items():
    avg_degree = sum(degrees.values()) / len(degrees)
    average_node_degrees[layer] = avg_degree

degree_threshold_counts = []
for layer, layer_graph in layer_graphs.items(): 
    degrees = node_degrees.get(layer, {})
    avg_degree = average_node_degrees.get(layer, 0)
    count = sum(1 for degree in degrees.values() if degree >= avg_degree)#节点度>平均度-1   向下取整
    degree_threshold_counts.append(count)

total_degree_threshold_count = sum(degree_threshold_counts)
normalized_degree_threshold_counts = [count / total_degree_threshold_count for count in degree_threshold_counts]
normalized_node_degree_counts_array = np.array(normalized_degree_threshold_counts)

average_shortest_path_lengths = {}
for layer, layer_graph in layer_graphs.items():
    shortest_path_lengths = dict(nx.shortest_path_length(layer_graph, weight='weight'))
    total_length = 0
    total_pairs = 0
    for node1 in layer_graph.nodes():
        for node2 in layer_graph.nodes():
            if node1 != node2:
                if node2 not in shortest_path_lengths.get(node1, {}):
                    length = 0
                else:
                    length = shortest_path_lengths[node1][node2]
                total_length += length
                total_pairs += 1
    
    avg_shortest_path_length = total_length / total_pairs if total_pairs > 0 else 0
    average_shortest_path_lengths[layer] = avg_shortest_path_length
    
total_avg_shortest_path_length = sum(average_shortest_path_lengths.values())
normalized_avg_shortest_path_lengths = []

for layer, layer_graph in layer_graphs.items(): 
    length = average_shortest_path_lengths.get(layer, 0)
    normalized_length = length / total_avg_shortest_path_length
    normalized_avg_shortest_path_lengths.append(normalized_length)

normalized_avg_shortest_path_lengths_array = np.array(normalized_avg_shortest_path_lengths)
num_layers = len(layer_graphs)
jaccard_coefficients = {}

for layer in range(1, num_layers + 1):  
    total_jaccard_coeff = 0
    for other_layer in range(1, num_layers + 1): 
        if layer != other_layer:
            intersection_size = len(set(layer_graphs[layer].edges()).intersection(layer_graphs[other_layer].edges()))
            union_size = len(set(layer_graphs[layer].edges()).union(layer_graphs[other_layer].edges()))
            jaccard_coeff = intersection_size / union_size if union_size > 0 else 0
            total_jaccard_coeff += jaccard_coeff
    jaccard_coefficients[layer] = total_jaccard_coeff

total_jaccard_coeff_sum = sum(jaccard_coefficients.values())
normalized_jaccard_coefficients = [coeff / total_jaccard_coeff_sum for coeff in jaccard_coefficients.values()]
normalized_jaccard_coefficients_array = np.array(normalized_jaccard_coefficients)

total_array = normalized_jaccard_coefficients_array + normalized_avg_shortest_path_lengths_array + normalized_node_degree_counts_array
max_node_id = max(node_id for layer_graph in layer_graphs.values() for edge in layer_graph.edges() for node_id in edge)
all_nodes = list(range(max_node_id + 1))

k_shell_values = {}
for layer, layer_graph in layer_graphs.items():
    layer_graph.remove_edges_from(nx.selfloop_edges(layer_graph))
    k_shell = nx.core_number(layer_graph)
    k_shell_values[layer] = {node: k_shell.get(node, 0) for node in all_nodes}

betweenness_centralities = {}
for layer, layer_graph in layer_graphs.items():
    betweenness_centrality = nx.betweenness_centrality(layer_graph)
    betweenness_centralities[layer] = {node: betweenness_centrality.get(node, 0) for node in all_nodes}
normalized_k_shell_values = {}

for layer in k_shell_values:
    unique_k_shell_values = list(set(k_shell_values[layer].values()))  
    total_k_shell = sum(unique_k_shell_values)
    normalized_k_shell_values[layer] = {node: k_shell_values[layer][node] / total_k_shell 
                                         for node in all_nodes if k_shell_values[layer][node] in unique_k_shell_values}
normalized_betweenness_centralities = {}

for layer in betweenness_centralities:
    unique_betweenness_values = list(set(betweenness_centralities[layer].values()))
    total_betweenness = sum(unique_betweenness_values)
    
    if total_betweenness > 0:
        normalized_betweenness_centralities[layer] = {node: betweenness_centralities[layer][node] / total_betweenness 
                                                      for node in all_nodes if betweenness_centralities[layer][node] in unique_betweenness_values}
    else:
        normalized_betweenness_centralities[layer] = {node: 0 for node in all_nodes}
layer_weights = total_array
num_layers = len(layer_graphs)

node_values = {}

for node in all_nodes:
    value = sum(
        (normalized_k_shell_values[layer][node] + normalized_betweenness_centralities[layer][node]) * layer_weights[layer - 1]
        for layer in range(1, num_layers + 1)
        if node in normalized_k_shell_values[layer] and node in normalized_betweenness_centralities[layer]
    )
    node_values[node] = value

node_values_array = np.array(list(node_values.values()))
print("Node Values Array (Me):")
result_string = ','.join(map(str, node_values_array))
print(result_string)